# Import required libraries

In [1]:
# pip install seaborn
# pip install xgboost
# pip install keras
# pip install sklearn
# pip install pandas
# pip install numpy

In [2]:
import pandas as pd

In [3]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


# Import the data

In [4]:
# help(pd.read_excel)

In [5]:
# df_x = pd.read_excel('Research data sample.xlsx', sheet_name = 'Sheet1')

In [6]:
df_x = pd.read_excel('Research data.xlsx', sheet_name = 'Sheet2')

In [7]:
df_x.head(2)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
df_x.columns

Index(['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18',
       'Column19', 'Column20', 'Column21'],
      dtype='object')

In [9]:
# df_y = pd.read_excel('Research data sample.xlsx', sheet_name = 'Sheet2')

In [11]:
df_y = pd.read_excel('Research data.xlsx', sheet_name = 'Sheet3')

In [12]:
df_y.head(2)

,Column1
0,66.193
1,66.193


In [13]:
df_y = df_y.rename(columns={'Column1': 'Position'})

In [14]:
df_y.head(2)

,Position
0,66.193
1,66.193


In [15]:
df_all = df_x
df_all['Position'] = df_y['Position']

In [16]:
# df_all.to_csv('all-combined-data.csv', index = False)

In [17]:
len(df_x)

729633

In [18]:
train_x = df_x[0:500000]
test_x = df_x[600000:]
train_y = df_y[0:500000]
test_y = df_y[600000:]

In [19]:
val_x = df_x[500000:600000]
val_y = df_y[500000:600000]

In [20]:
# df_all.head(2)

In [21]:
# from sklearn.model_selection import train_test_split

# train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [22]:
train_x.head(2)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Position
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,66.193
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,66.193


In [23]:
len(test_x)

129633

In [24]:
train_x.shape[1]

22

# Training a Simple Linear Regression Model

#### Followed this blogpost: https://aigeekprogrammer.com/binary-classification-using-logistic-regression-and-keras/

In [35]:
LR_model = Sequential()

# The Input Layer :
LR_model.add(Dense(1, kernel_initializer='normal',input_dim = train_x.shape[1], activation='linear'))

# Compile the network :
LR_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
LR_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 23        
Total params: 23
Trainable params: 23
Non-trainable params: 0
_________________________________________________________________


#### Create a folder call "lr_models" to save your model weights

In [36]:
checkpoint_name = 'lr_models/Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [37]:
callbacks_list

In [38]:
train = train_x.to_numpy()

In [39]:
train.shape

(500000, 22)

In [40]:
target = train_y.to_numpy()

In [41]:
target.shape

(500000, 1)

In [42]:
# ?NN_model.fit

In [43]:
val_y.to_numpy().shape

(100000, 1)

In [44]:
LR_model.fit(train, target, epochs=50, batch_size=64, validation_data = (val_x.to_numpy(), val_y.to_numpy()), callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Train on 500000 samples, validate on 100000 samples
Epoch 1/50
500000/500000 [==============================] - 12s 23us/step - loss: 12.9852 - mean_absolute_error: 12.9852 - val_loss: 0.0280 - val_mean_absolute_error: 0.0280

Epoch 00001: val_loss improved from inf to 0.02796, saving model to lr_models/Weights-001--0.02796.hdf5
Epoch 2/50
500000/500000 [==============================] - 11s 22us/step - loss: 0.0205 - mean_absolute_error: 0.0205 - val_loss: 0.0297 - val_mean_absolute_error: 0.0297

Epoch 00002: val_loss did not improve from 0.02796
Epoch 3/50
500000/500000 [==============================] - 11s 22us/step - loss: 0.0205 - mean_absolute_error: 0.0205 - val_loss: 0.0348 - val_mean_absolute_error: 0.0348

Epoch 00003: val_loss did not improve from 0.02796
Epoch 4/50
500000/500000 [==============================] - 12s 23us/step - loss: 0.0205 - mean_absolute_error: 0.0205 - val_loss: 0.0253 - val_mean_absolute_error: 0.0253



500000/500000 [==============================] - 11s 22us/step - loss: 0.0220 - mean_absolute_error: 0.0220 - val_loss: 0.0100 - val_mean_absolute_error: 0.0100

Epoch 00035: val_loss did not improve from 0.00063
Epoch 36/50
500000/500000 [==============================] - 12s 24us/step - loss: 0.0207 - mean_absolute_error: 0.0207 - val_loss: 0.0198 - val_mean_absolute_error: 0.0198

Epoch 00036: val_loss did not improve from 0.00063
Epoch 37/50
500000/500000 [==============================] - 11s 22us/step - loss: 0.0208 - mean_absolute_error: 0.0208 - val_loss: 0.0204 - val_mean_absolute_error: 0.0204

Epoch 00037: val_loss did not improve from 0.00063
Epoch 38/50
500000/500000 [==============================] - 10s 20us/step - loss: 0.0210 - mean_absolute_error: 0.0210 - val_loss: 0.0218 - val_mean_absolute_error: 0.0218

Epoch 00038: val_loss did not improve from 0.00063
Epoch 39/50
500000/500000 [==============================] - 10s 20us/step - loss: 0.0208 - mean_absolute_error:

In [46]:
wights_file = 'lr_models/Weights-006--0.00063.hdf5' # choose the best checkpoint 
LR_model.load_weights(wights_file) # load it
LR_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [47]:
test = test_x.to_numpy()

In [49]:
predictions = LR_model.predict(test)

In [50]:
predictions[1]

array([215.34814], dtype=float32)

In [51]:
actual = test_y.to_numpy()

In [52]:
actual[1]

array([215.35])

### We will judge our model on Mean Absolute Error Metric

In [53]:
MAE = mean_absolute_error(actual , predictions)
print('Deep Neural Network validation MAE = ', MAE)

Deep Neural Network validation MAE =  0.0006395305843977815
